In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# install onnx libraries

# !pip install onnx
# !pip install onnx_tf
# !pip install onnx2keras

## Saving and Loading PyTorch Model

In [3]:
import torch
import torchvision.models as models

# initializing a model with random weights
model = models.resnet18()

# initializing a model with pre-trained weights
model = models.resnet18(pretrained=True)

In a PyTorch model file, complete model is stored while in a state file, only parameters are stored.

### 1. Saving state

In [4]:
# saving the model state
# extension .pt/.pth
torch.save(model.state_dict(), 'pytorch_weights.pth')

# loading the model
# initializing model with random weights
model = models.resnet18()

# loading the state dict
model.load_state_dict(torch.load('pytorch_weights.pth'))

<All keys matched successfully>

### 2. Saving model

In [5]:
# saving model
torch.save(model,'pytorch_model.pth')

# loading model
model = torch.load('pytorch_model.pth')

## Saving and Loading Keras Model

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50

# instantiating a model
model = ResNet50(weights='imagenet')

### 1. HDF5 format

In [7]:
import json

# saving weights
# serialize weights to HDF5
model.save_weights('keras_weights.h5')

# serialize model to JSON
# json.loads takes string as input and returns dictionary as output
model_json = json.loads(model.to_json())
with open('keras_model.json', 'w') as f:
    json.dump(model_json, f, indent=4)

In [8]:
from keras.models import model_from_json

# loading model
# deserialize JSON to model
with open('keras_model.json') as f:
    model_json = json.load(f)

# json.dumps takes dictionary as input and returns string as output 
model = model_from_json(json.dumps(model_json))

# loading weights
# deserialize HDF5 to model
model.load_weights('keras_weights.h5')

### 2. Protobuf (.pb) format

In [9]:
from keras.models import load_model

# saving a model
model.save('saved_model')

# loading a model
model = load_model('saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


## PyTorch to ONNX (Open Neural Network Exchange)

In [10]:
import torch
import torchvision

# loading pytorch model
model = torch.load('pytorch_model.pth')
dummy_input = torch.randn([10,3,244,244])

# exporting model to ONNX format
torch.onnx.export(model, dummy_input, 'resnet.onnx', verbose=False)

## ONNX to Tensorflow/Keras

In [11]:
import onnx
from onnx_tf.backend import prepare
from onnx2keras import onnx_to_keras

# onnx to tensorflow model
model = onnx.load('resnet.onnx')
tf_model = prepare(model)

# onnx to keras model
keras_model = onnx_to_keras(model, input_names = [node.name for node in model.graph.input], verbose=False)

# save keras model
# save weights
keras_model.save_weights('onnx2keras_weights.h5')

# serialize model to JSON
model_json = json.loads(keras_model.to_json())
with open('onnx2keras_model.json', 'w') as f:
    json.dump(model_json, f, indent=4)

Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.


## Keras to TensorFlow Lite (TFLite)

In [12]:
import json
import tensorflow as tf
from keras.models import model_from_json

# loading keras model
with open('keras_model.json') as f:
    model_json = json.load(f)
model = model_from_json(json.dumps(model_json))
model.load_weights('keras_weights.h5')

# tflite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# optimizations
optimize_for_speed = False
optimize_for_size = False

if optimize_for_speed:
    converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
elif optimize_for_size:
    converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
else:
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

# convert keras model to tflite
tflite_model = converter.convert()

# saving model
with open('keras2tflite.tflite','wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/88/brn_pbx106183309rb29k7lh0000gq/T/tmpxj0mt1ya/assets


In [13]:
# load model and allocate tensors
tflite_model = tf.lite.Interpreter('keras2tflite.tflite')
tensor_details = tflite_model.get_tensor_details()
tflite_model.allocate_tensors()

input_details = tflite_model.get_input_details()[0]
output_details = tflite_model.get_output_details()[0]

# input details
print('INPUT DETAILS')
print(input_details)

# output details
print('\nOUTPUT DETAILS')
print(output_details)

INPUT DETAILS
{'name': 'input_1', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

OUTPUT DETAILS
{'name': 'Identity', 'index': 186, 'shape': array([   1, 1000], dtype=int32), 'shape_signature': array([  -1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


### TFLite to Keras 

Since the model was optimized while converting Keras model to TFLite, and the weights were quantized potentially leading to loss of precision, converting TFLite model back to Keras might not lead to the same Keras model.

The TFLite model contains information of weights which can be extracted. However, information of loss function and optimizer might not be available since it's not required during inference.

In [14]:
weight_dict = {}

# extract weights from tflite model
for tensor in tensor_details:
    name = tensor['name']
    weights = tflite_model.tensor(tensor['index'])()
    weight_dict[name] = weights